In [145]:
import nltk
import pandas as pd
from nltk.corpus import stopwords
import re
import numpy as np
from nltk.corpus import stopwords
from nltk import word_tokenize
from sklearn.metrics import classification_report
import fasttext
from collections import Counter
import pandas as pd
import numpy as np 
import re
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from collections import Counter
import random

In [146]:
STOPWORDS = set(stopwords.words('english_amz'))
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
LINKS =re.compile('(?i)(http(s)?[:/.a-z0-9]*)')
CHAR_OCC = re.compile(r'(.)\1{2,}')
LINKS = re.compile("(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))")
def preprocess_tweet(text):
    text = text.lower() # lowercase text
    text = LINKS.sub('', text)
    text = CHAR_OCC.sub(r'\1',text) # replace more than 2 occurences of a same char like aaaaabbbbbbb to ab
    #text = ' '.join(word for word in text.split() if word not in STOPWORDS) # delete stopwors from text
    text = re.sub("@[a-zA-Z]*", "", text) # remove usernames with @******
    text = BAD_SYMBOLS_RE.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text
    text = re.sub("\'", "", text) # remove backslash-apostrophe 
    text = re.sub("[^a-zA-Z]"," ",text) # remove everything except alphabets
    text = re.sub("( |^)[a-zA-Z][a-zA-Z]? "," ",text) # remove words with just 1 or 2 letters
    text = ' '.join(text.split()) # remove whitespaces 
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # delete stopwors from text
    return text

## Train data
df_train = pd.read_excel('new_data/hasoc_2020_de_train_new.xlsx')
df_train['text'] = df_train['text'].apply(preprocess_tweet)
# adding duplicates
df_train['Quantity'] = df_train['task1'].apply(lambda x: 1 if x=='NOT' else random.randint(3,4))
df_train = pd.DataFrame([df_train.ix[idx] 
                       for idx in df_train.index 
                       for _ in range(df_train.ix[idx]['Quantity'])]).reset_index(drop=True)

df_train['needed'] = df_train['task1'].apply(lambda x: '__label__'+x)
header = ["text", "needed"]
df_train.to_csv('train.txt', columns = header,index=False,sep = " ")

## Test data
df_test = pd.read_csv('test/german_test_1509.csv')
df_test['text'] = df_test['text'].apply(preprocess_tweet)
df_test['needed'] = df_test['task1'].apply(lambda x: '__label__'+x)
header = ["text", "needed"]
df_test.to_csv('test.txt', columns = header,index=False,sep = " ")

/home/pj/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:27: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


In [147]:
# Train final classifiers
classifier1 = fasttext.train_supervised('train.txt', lr=0.1, wordNgrams=1, epoch=20)
classifier2 = fasttext.train_supervised('train.txt', lr=0.1, wordNgrams=2, epoch=20)
classifier3 = fasttext.train_supervised('train.txt', lr=0.1, wordNgrams=3, epoch=20)

# Predict test data
predictions1 = classifier1.predict(df_test.text.tolist())
predictions2 = classifier2.predict(df_test.text.tolist())
predictions3 = classifier3.predict(df_test.text.tolist())

# Combine predictions
majority_vote = np.array([])
for i in range(len(predictions1[0])):
    majority_vote = np.append(majority_vote, Counter([predictions1[0][i][0],
                                                   predictions2[0][i][0],
                                                   predictions3[0][i][0]]).most_common(1)[0][0])
    
majority_vote_upd = [x.replace("__label__","") for x in majority_vote]
print(classification_report(df_test['task1'],majority_vote_upd))

              precision    recall  f1-score   support

         HOF       0.72      0.66      0.69       134
         NOT       0.89      0.91      0.90       392

    accuracy                           0.85       526
   macro avg       0.80      0.78      0.79       526
weighted avg       0.84      0.85      0.85       526



In [148]:
classifier1 = fasttext.train_supervised('train.txt', lr=0.1, wordNgrams=2, epoch=40)
predictions1 = classifier1.predict(df_test.text.tolist())
majority_vote_upd = [x[0].replace("__label__","") for x in predictions1[0]]
print(classification_report(df_test['task1'],majority_vote_upd))

              precision    recall  f1-score   support

         HOF       0.72      0.65      0.69       134
         NOT       0.88      0.92      0.90       392

    accuracy                           0.85       526
   macro avg       0.80      0.78      0.79       526
weighted avg       0.84      0.85      0.85       526



In [101]:
df_train['labels_text'] = '__label__' + df_train.task1 
df_train.labels_text = df_train.labels_text.str.cat(df_train.text, sep=' ')

# Function to do K-fold CV across different fasttext parameter values
def tune(Y, X, YX, k, lr, wordNgrams, epoch):
    # Record results
    results = []
    for lr_val in lr:
        for wordNgrams_val in wordNgrams:
            for epoch_val in epoch:  
                # K-fold CV
                kf = KFold(n_splits=k, shuffle=True)
                fold_results = []
                # For each fold
                for train_index, test_index in kf.split(X):
                    # Write the training data for this CV fold
                    training_file = open('train_cv.txt','w')
                    training_file.writelines(YX[train_index] + '\n')
                    training_file.close()
                    # Fit model for this set of parameter values
                    model = fasttext.train_supervised('train_cv.txt',
                                          lr=lr_val,
                                          wordNgrams=wordNgrams_val,
                                          epoch=epoch_val)
                    # Predict the holdout sample
                    pred = model.predict(X[test_index].tolist())
                    pred = pd.Series(pred[0]).apply(lambda x: re.sub('__label__', '', x[0]))
                    # Compute accuracy for this CV fold
                    fold_results.append(accuracy_score(Y[test_index], pred.values))
                # Compute mean accuracy across 10 folds
                mean_acc = pd.Series(fold_results).mean()
                print([lr_val, wordNgrams_val, epoch_val, mean_acc])
    # Add current parameter values and mean accuracy to results table
    results.append([lr_val, wordNgrams_val, epoch_val, mean_acc])         
    # Return as a DataFrame 
    results = pd.DataFrame(results)
    results.columns = ['lr','wordNgrams','epoch','mean_acc']
    return(results)
    
#Tune parameters using 10-fold CV
results = tune(Y = df_train.task1,
    X = df_train.text,
    YX = df_train.labels_text,
    k = 10, 
    lr = [0.05, 0.1, 0.2],
    wordNgrams = [1,2,3],
    epoch = [15,17,20])

[0.05, 1, 15, 0.8805208712755883]
[0.05, 1, 17, 0.8781139360384644]
[0.05, 1, 20, 0.8740547825453486]
[0.05, 2, 15, 0.8797217163254899]
[0.05, 2, 17, 0.8789058060756174]
[0.05, 2, 20, 0.8767458293873387]
[0.05, 3, 15, 0.8724353463976104]
[0.05, 3, 17, 0.875407590879289]
[0.05, 3, 20, 0.8767552997741677]
[0.1, 1, 15, 0.869198659576018]
[0.1, 1, 17, 0.8713695636337146]
[0.1, 1, 20, 0.8721716325489911]
[0.1, 2, 15, 0.8791731623807095]
[0.1, 2, 17, 0.8818802360311795]
[0.1, 2, 20, 0.8791819042762439]
[0.1, 3, 15, 0.8770204706053762]
[0.1, 3, 17, 0.8767545712828732]
[0.1, 3, 20, 0.8775486267939098]
[0.2, 1, 15, 0.8686610330006557]
[0.2, 1, 17, 0.8640824652145407]
[0.2, 1, 20, 0.8689342172361041]
[0.2, 2, 15, 0.8797282727471407]
[0.2, 2, 17, 0.8783725504480222]
[0.2, 2, 20, 0.8764959568733154]
[0.2, 3, 15, 0.8770270270270271]
[0.2, 3, 17, 0.876750200335106]
[0.2, 3, 20, 0.8805354411014787]


In [140]:
classifier1.predict('sdf')

(('__label__NOT',), array([0.52291888]))

In [141]:
print(classifier1.words)
print(classifier1.labels)

['</s>', 'hai', 'hai"', 'india', '"httpstco', 'bjp', 'httpstco', 'sharma', '"india', '"bjp', 'swaraj', 'singh', 'https"', 'masoodazhar', 'ind', 'shri', 'asaram', 'https', 'happymothersday', 'unsc', 'evm', 'kabir"', 'vote', 'chutiya', 'india"', 'khan', 'aap', 'tyagi"', 'q"', 'kya', 'k"', '"ind', '"masoodazhar', 'kar', '"anil', '"sant', '"allah', 'mein', 'httpstco"', 'gujarat', 'sab', 'indianjudiciaryatstake', 'bapu', 'nahi', '"sharma', '"bharat', 'banaya', 'bjp"', 'h"', 'aap"', '"loksabhaelections', 'uditraj', 'karta', 'rss', 'meravotemodiko', 'tmc', 'ajay', 'summerhealthtipsbybapuji', 'dailyhunt', 'main', 'evm"', 'news', '"gst', '"httpstcou', 'inch', 'votinground"', 'ipl', 'u"', 'news"', 'amp', 'piyarye', 'r"', 'saturdaymotivation', 'naam', 'rizvi', '"modi', 'rathee', 'wednesdaywisdom', 'namo', 'loksabhaelections', 'kya"', '"evm', 'humittal', 'suhana', 'inc', '"radar', 'nota', 'aur', 'hindu', '"breaking', 'oh"', 'httpst"', 'summer"', '"hot', 'sakta', 'pramod', 'gst', 'httpstcog', 'par'